In [4]:
import pandas as pd
import csv
import base64
import datetime
import io
import time
from time import mktime
import sys
import numpy as np

import matplotlib.pyplot as plt
import plotly
import chart_studio.plotly as py
import seaborn as sns
from plotly import graph_objs as go
import plotly.express as px
import dash
import dash_table
from datetime import date
from dash import Dash, dcc, html
from dash.dependencies import Input, State, Output
import dash_bootstrap_components as dbc
from dash import callback_context

import base64
import os
from urllib.parse import quote as urlquote

from flask import Flask, send_from_directory
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, State, Output
import dash_bootstrap_components as dbc
from fractions import Fraction
from math import ceil

import matplotlib.pyplot as plt

import requests
import csv
import json
from urllib.request import urlopen

import openpyxl
from openpyxl import Workbook, load_workbook

import math


C:\Users\NADIAJ~1\AppData\Local\Temp/ipykernel_9836/104781541.py:18: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table
C:\Users\NADIAJ~1\AppData\Local\Temp/ipykernel_9836/104781541.py:31: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\NADIAJ~1\AppData\Local\Temp/ipykernel_9836/104781541.py:32: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


# Access Production API & DATA Conversion

In [5]:
production_url='http://10.12.13.164:8110/api/BasicQMSData/BirichinaDataForReport?StartDate=2022-04-12&EndDate=2022-04-12'
response = urlopen(production_url)
production_data_json = json.loads(response.read())
productionapi=pd.DataFrame(production_data_json)


In [6]:
raw_api=productionapi.copy()
########### Convert Time ###############
raw_api['Time']= pd.to_datetime(raw_api['Time'])
raw_api['Time']=raw_api['Time'].apply(lambda t: t.strftime('%H'))
raw_api['Time']=raw_api['Time'].astype(str)
########## Convert BatchQTY ############
raw_api['BatchQty']=raw_api['BatchQty'].astype(int)
########## Convert DefectCount ############
raw_api['DefectCount']=raw_api['DefectCount'].astype(int)
########## Convert SMV ############
raw_api['SMV']=raw_api['SMV'].astype(float)
########## Convert LineNumber Into Integer #########
raw_api['LineNumber'] = raw_api['LineNumber'].str.replace('Line-','').astype(int)


raw_api=raw_api.fillna(0)
raw_api2=raw_api[['BusinessUnit', 'LineNumber','Time', 'Date','GarmentsNumber',
       'DefectName', 'DefectType', 'DefectCount']]
raw_api2

,BusinessUnit,LineNumber,Time,Date,GarmentsNumber,DefectName,DefectType,DefectCount
0,Unit-1,1,09,12/04/2022,805691003093249,na,Ok,0
1,Unit-1,1,09,12/04/2022,805691007093250,na,Ok,0
2,Unit-1,1,09,12/04/2022,805691030093250,na,Ok,0
3,Unit-1,1,09,12/04/2022,805691034093250,na,Ok,0
4,Unit-1,1,09,12/04/2022,805692009093337,na,Ok,0
...,...,...,...,...,...,...,...,...
102727,Unit-4,27,14,12/04/2022,487293002140524,na,Ok,0
102728,Unit-4,27,14,12/04/2022,487294011140539,na,Ok,0
102729,Unit-4,27,14,12/04/2022,487295002140612,na,Ok,0
102730,Unit-4,27,14,12/04/2022,487295004140612,na,Ok,0


In [13]:
raw_api_calc=raw_api2.copy()

######################################## Total ############################################
mac=raw_api_calc.copy()
mac=mac.groupby(['Date','BusinessUnit','LineNumber']).agg({'GarmentsNumber':'nunique','DefectCount' :'sum'}).reset_index()
mac['Total Defect']=mac['DefectCount']
mac['Total Garments']=mac['GarmentsNumber']
del mac['DefectCount']
del mac['GarmentsNumber']
mac

############################################## Counting Ok Defect type ############################
mav=raw_api_calc.copy()
mav=mav[mav['DefectType']=='Ok']
mav=mav.groupby(['Date','BusinessUnit','LineNumber']).agg({'DefectCount' :'sum','DefectType':'count'}).reset_index()
mav['Ok']=mav['DefectType']
del mav['DefectType']
del mav['DefectCount']
mav
############################################## Counting Reject Defect type ############################
mav1=raw_api_calc.copy()
mav1=mav1[mav1['DefectType']=='Reject']
mav1=mav1.groupby(['Date','BusinessUnit','LineNumber']).agg({'DefectCount' :'sum','DefectType':'count'}).reset_index()
mav1['Reject']=mav1['DefectType']
del mav1['DefectType']
del mav1['DefectCount']
mav1
############################################## Counting Rework Defect type ############################
mav2=raw_api_calc.copy()
mav2=mav2[mav2['DefectType']=='Rework']
mav2=mav2.groupby(['Date','BusinessUnit','LineNumber']).agg({'DefectCount' :'sum','DefectType':'count'}).reset_index()
mav2['Rework']=mav2['DefectType']
del mav2['DefectType']
del mav2['DefectCount']
mav2

mao=pd.merge(mac,mav,on=['Date','BusinessUnit','LineNumber'])
mao
mao1=pd.merge(mao,mav1,on=['Date','BusinessUnit','LineNumber'],how='left')
mao1
mao2=pd.merge(mao1,mav2,on=['Date','BusinessUnit','LineNumber'],how='left')

mao2=mao2.fillna(0)

mao2['NewTotalGarments']=mao2['Total Garments']-(mao2['Reject']+mao2['Rework'])
mao2['Reject']=mao2['Reject'].astype(int)
mao2['Rework']=mao2['Rework'].astype(int)
mao2['NewTotalGarments']=mao2['NewTotalGarments'].astype(int)
del mao2['NewTotalGarments']
mao2=mao2[['Date','BusinessUnit','LineNumber','Total Defect','Total Garments','Ok', 'Reject', 'Rework']]
mao2

prod_time=raw_api.copy()

prod_time=prod_time.groupby(['Date','BusinessUnit', 'LineNumber','Time']).agg({'GarmentsNumber':'nunique'}).reset_index()
prod_time= prod_time.pivot_table('GarmentsNumber', ['Date','BusinessUnit', 'LineNumber'], 'Time').reset_index()
prod_time=prod_time.fillna(0)

prod_time['6am-7am']  =prod_time['06'].astype(int)
prod_time['7am-8am']  =prod_time['07'].astype(int)
prod_time['8am-9am']  =prod_time['08'].astype(int)
prod_time['9am-10am'] =prod_time['09'].astype(int)
prod_time['10am-11am']=prod_time['10'].astype(int)
prod_time['11am-12am']=prod_time['11'].astype(int)
prod_time['12pm-1pm'] =prod_time['12'].astype(int)
prod_time['1pm-2pm']  =prod_time['13'].astype(int)
prod_time['2pm-3pm']  =prod_time['14'].astype(int)
prod_time['3pm-4pm']  =prod_time['15'].astype(int)
prod_time['4pm-5pm']  =prod_time['16'].astype(int)
prod_time['5pm-6pm']  =prod_time['17'].astype(int)
# prod_time['6pm-7pm']  =prod_time['18'].astype(int)
prod_time['8pm-9pm']  =prod_time['19'].astype(int)
prod_time['9pm-10pm'] =prod_time['20'].astype(int)
# prod_time['11pm-12pm']=prod_time['21'].astype(int)

del prod_time['06']
del prod_time['07']
del prod_time['08']
del prod_time['09']
del prod_time['10']
del prod_time['11']
del prod_time['12']
del prod_time['13']
del prod_time['14']
del prod_time['15']
del prod_time['16']
del prod_time['17']
# del prod_time['18']
del prod_time['19']
del prod_time['20']
# del prod_time['21']

prod_merge=pd.merge(mao2,prod_time,on=['Date','BusinessUnit', 'LineNumber'])
prod_merge

,Date,BusinessUnit,LineNumber,Total Defect,Total Garments,Ok,Reject,Rework,6am-7am,7am-8am,...,10am-11am,11am-12am,12pm-1pm,1pm-2pm,2pm-3pm,3pm-4pm,4pm-5pm,5pm-6pm,8pm-9pm,9pm-10pm
0,12/04/2022,Unit-1,1,0,6200,6200,0,0,0,0,...,610,3574,840,0,0,180,636,0,0,0
1,12/04/2022,Unit-1,2,5,3654,3649,1,0,100,149,...,544,300,50,246,200,246,1372,0,0,0
2,12/04/2022,Unit-1,9,0,222,222,0,0,0,0,...,0,222,0,0,0,0,0,0,0,0
3,12/04/2022,Unit-1,13,52,5478,5462,0,9,0,523,...,805,620,1213,0,839,481,0,0,0,0
4,12/04/2022,Unit-1,14,0,4768,4768,0,0,0,156,...,0,890,100,1170,1462,492,50,0,0,0
5,12/04/2022,Unit-1,15,0,4620,4620,0,0,0,0,...,294,210,210,126,462,0,210,0,0,0
6,12/04/2022,Unit-1,17,0,420,420,0,0,0,0,...,42,378,0,0,0,0,0,0,0,0
7,12/04/2022,Unit-1,19,0,842,841,0,1,0,0,...,0,301,0,0,0,541,0,0,0,0
8,12/04/2022,Unit-1,21,0,1467,1467,0,0,0,0,...,0,0,0,0,0,844,623,0,0,0
9,12/04/2022,Unit-1,23,0,96,96,0,0,0,0,...,0,0,0,0,0,0,96,0,0,0


# Forecast API & Data Conversion

In [14]:
Forecast_excel =pd.read_excel("C:\\Users\\nadiajebin\\Desktop\\Hourly Dashboard\\Final\\Forecast_10th_April.xlsx")
forecast=Forecast_excel[['ProductionUnit','LineNumber', 'Date','PresentCadre','ForecastPcs',
                   'ForecastSAH','ForecastEff','Working_Hour','SMV']]
forecast['LineNumber'] = forecast['LineNumber'].str.replace('Line-','').astype(int)
forecast['Date']=forecast['Date'].dt.strftime('%d/%m/%Y')
forecast4apr=forecast.copy()
forecast4apr['BusinessUnit']=forecast4apr['ProductionUnit']
del forecast4apr['ProductionUnit']
forecast4apr=forecast4apr[['BusinessUnit','LineNumber','Date','PresentCadre','ForecastPcs',
                           'ForecastSAH','ForecastEff','Working_Hour','SMV']]
forecast4apr=forecast4apr.fillna(0)
forecast4apr=forecast4apr.round(2)
forecast4apr

C:\Users\NADIAJ~1\AppData\Local\Temp/ipykernel_9836/1415248643.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast['LineNumber'] = forecast['LineNumber'].str.replace('Line-','').astype(int)
C:\Users\NADIAJ~1\AppData\Local\Temp/ipykernel_9836/1415248643.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast['Date']=forecast['Date'].dt.strftime('%d/%m/%Y')


,BusinessUnit,LineNumber,Date,PresentCadre,ForecastPcs,ForecastSAH,ForecastEff,Working_Hour,SMV
0,Unit-2,1,04/04/2022,24,2497.0,207.25,86,10,4.98
1,Unit-2,2,04/04/2022,25,4044.0,192.09,76,10,2.85
2,Unit-2,3,04/04/2022,24,3324.0,214.95,89,10,3.88
3,Unit-2,4,04/04/2022,25,4349.0,216.00,86,10,2.98
4,Unit-2,5,04/04/2022,25,3137.0,149.01,59,10,2.85
...,...,...,...,...,...,...,...,...,...
765,Unit-4,23,10/04/2022,29,700.0,176.40,55,10,15.12
766,Unit-4,24,10/04/2022,26,1000.0,209.00,73,10,12.54
767,Unit-4,25,10/04/2022,31,880.0,208.56,61,10,14.22
768,Unit-4,26,10/04/2022,26,1100.0,211.02,73,10,11.51


# Forecast Production Merge

In [15]:
Prod_Fore2=pd.merge(forecast4apr, prod_merge,on=['BusinessUnit','LineNumber','Date'],how='outer')
Prod_Fore2=Prod_Fore2.fillna(0)
Prod_Fore2

Prod_Fore2
Prod_Fore3=Prod_Fore2.copy()

# Calculation

In [16]:
Prod_Fore2['Actual SAH']=(Prod_Fore2['SMV']*Prod_Fore2['Ok'])/60
Prod_Fore2['Actual EFF']=(Prod_Fore2['Actual SAH']*100)/(Prod_Fore2['Working_Hour']*Prod_Fore2['PresentCadre'])
Prod_Fore2=Prod_Fore2.round(2)
Prod_Fore2=Prod_Fore2.fillna(0)
Prod_Fore2

,BusinessUnit,LineNumber,Date,PresentCadre,ForecastPcs,ForecastSAH,ForecastEff,Working_Hour,SMV,Total Defect,...,12pm-1pm,1pm-2pm,2pm-3pm,3pm-4pm,4pm-5pm,5pm-6pm,8pm-9pm,9pm-10pm,Actual SAH,Actual EFF
0,Unit-2,1,04/04/2022,24.0,2497.0,207.25,86.0,10.0,4.98,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Unit-2,2,04/04/2022,25.0,4044.0,192.09,76.0,10.0,2.85,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Unit-2,3,04/04/2022,24.0,3324.0,214.95,89.0,10.0,3.88,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Unit-2,4,04/04/2022,25.0,4349.0,216.00,86.0,10.0,2.98,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Unit-2,5,04/04/2022,25.0,3137.0,149.01,59.0,10.0,2.85,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
811,Unit-3,28,12/04/2022,0.0,0.0,0.00,0.0,0.0,0.00,87.0,...,192.0,48.0,770.0,2519.0,252.0,0.0,0.0,0.0,0.0,0.0
812,Unit-3,30,12/04/2022,0.0,0.0,0.00,0.0,0.0,0.00,0.0,...,96.0,0.0,0.0,816.0,0.0,0.0,0.0,0.0,0.0,0.0
813,Unit-4,12,12/04/2022,0.0,0.0,0.00,0.0,0.0,0.00,0.0,...,0.0,0.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
814,Unit-4,16,12/04/2022,0.0,0.0,0.00,0.0,0.0,0.00,14.0,...,0.0,0.0,336.0,0.0,494.0,0.0,0.0,0.0,0.0,0.0


Unit Wise without Plan

In [17]:
Prod_Fore3_sum=Prod_Fore2.groupby(['BusinessUnit','Date']).agg({'PresentCadre':'sum', 'ForecastPcs':'sum',
       'ForecastSAH':'sum','Working_Hour':'sum','Actual SAH':'sum', 'Total Defect':'sum',
       'Total Garments':'sum', 'Ok':'sum', 'Reject':'sum', 'Rework':'sum','6am-7am':'sum',
       '7am-8am':'sum', '8am-9am':'sum', '9am-10am':'sum', '10am-11am':'sum', '11am-12am':'sum', '12pm-1pm':'sum',
       '1pm-2pm':'sum', '2pm-3pm':'sum', '3pm-4pm':'sum', '4pm-5pm':'sum', '5pm-6pm':'sum', '8pm-9pm':'sum'}).reset_index()


Prod_Fore3_sum['Actual EFF']=(Prod_Fore3_sum['Actual SAH']*100)/(Prod_Fore3_sum['Working_Hour']*Prod_Fore3_sum['PresentCadre'])
#Prod_Fore3_sum['Forecast EFF']=(Prod_Fore3_sum['ForecastSAH']*100)/(Prod_Fore3_sum['Working_Hour']*Prod_Fore3_sum['PresentCadre'])
Prod_Fore3_sum['Forecast EFF']=(Prod_Fore3_sum['ForecastSAH']*100)/(24*10*26)
# Plan_Fore3_sum[]
Prod_Fore3_sum=Prod_Fore3_sum.round(2)
Prod_Fore4_sum=Prod_Fore3_sum.copy()
Prod_Fore4_sum=Prod_Fore4_sum[['BusinessUnit', 'Date', 'PresentCadre', 'Total Defect','Reject', 'Rework','Working_Hour',
               'ForecastPcs', 'ForecastSAH', 'Forecast EFF','Ok','Actual SAH','Actual EFF','6am-7am', '7am-8am', '8am-9am', '9am-10am',
               '10am-11am', '11am-12am', '12pm-1pm', '1pm-2pm', '2pm-3pm', '3pm-4pm','4pm-5pm', '5pm-6pm', '8pm-9pm']]
Prod_Fore4_sum=Prod_Fore4_sum.fillna(0)
Prod_Fore4_sum

,BusinessUnit,Date,PresentCadre,Total Defect,Reject,Rework,Working_Hour,ForecastPcs,ForecastSAH,Forecast EFF,...,9am-10am,10am-11am,11am-12am,12pm-1pm,1pm-2pm,2pm-3pm,3pm-4pm,4pm-5pm,5pm-6pm,8pm-9pm
0,Unit-1,03/04/2022,620.0,0.0,0.0,0.0,270.0,99780.00,4155.52,66.59,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Unit-1,04/04/2022,623.0,0.0,0.0,0.0,270.0,97630.00,4119.99,66.03,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Unit-1,05/04/2022,626.0,0.0,0.0,0.0,270.0,101876.00,4385.58,70.28,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Unit-1,06/04/2022,624.0,0.0,0.0,0.0,270.0,99630.00,4388.36,70.33,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Unit-1,07/04/2022,624.0,0.0,0.0,0.0,270.0,96260.00,4287.99,68.72,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Unit-1,10/04/2022,624.0,0.0,0.0,0.0,270.0,81350.00,3917.97,62.79,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Unit-1,12/04/2022,0.0,64.0,4.0,14.0,0.0,0.00,0.00,0.00,...,3234.0,2295.0,6495.0,2413.0,1542.0,3590.0,2784.0,3859.0,192.0,482.0
7,Unit-1,31/03/2022,616.0,0.0,0.0,0.0,324.0,119650.00,4961.87,79.52,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Unit-2,02/04/2022,635.0,0.0,0.0,0.0,260.0,68230.00,4430.56,71.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Unit-2,03/04/2022,635.0,0.0,0.0,0.0,260.0,71311.00,4453.04,71.36,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Line Wise without Plan

In [19]:
Prod_Fore2_line=Prod_Fore2[['BusinessUnit', 'LineNumber', 'Date', 'SMV','Working_Hour','PresentCadre','Total Defect',
                'Total Garments','Reject', 'Rework','ForecastPcs','ForecastSAH', 'ForecastEff',
                'Ok','Actual SAH', 'Actual EFF','6am-7am', '7am-8am',
       '8am-9am', '9am-10am', '10am-11am', '11am-12am', '12pm-1pm', '1pm-2pm',
       '2pm-3pm', '3pm-4pm', '4pm-5pm', '5pm-6pm', '8pm-9pm','9pm-10pm']]
Prod_Fore2_line
# '6pm-7pm', '11pm-12pm'

,BusinessUnit,LineNumber,Date,SMV,Working_Hour,PresentCadre,Total Defect,Total Garments,Reject,Rework,...,10am-11am,11am-12am,12pm-1pm,1pm-2pm,2pm-3pm,3pm-4pm,4pm-5pm,5pm-6pm,8pm-9pm,9pm-10pm
0,Unit-2,1,04/04/2022,4.98,10.0,24.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Unit-2,2,04/04/2022,2.85,10.0,25.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Unit-2,3,04/04/2022,3.88,10.0,24.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Unit-2,4,04/04/2022,2.98,10.0,25.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Unit-2,5,04/04/2022,2.85,10.0,25.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
811,Unit-3,28,12/04/2022,0.00,0.0,0.0,87.0,4631.0,0.0,0.0,...,526.0,36.0,192.0,48.0,770.0,2519.0,252.0,0.0,0.0,0.0
812,Unit-3,30,12/04/2022,0.00,0.0,0.0,0.0,1008.0,0.0,0.0,...,0.0,0.0,96.0,0.0,0.0,816.0,0.0,0.0,0.0,0.0
813,Unit-4,12,12/04/2022,0.00,0.0,0.0,0.0,37.0,0.0,0.0,...,0.0,0.0,0.0,0.0,37.0,0.0,0.0,0.0,0.0,0.0
814,Unit-4,16,12/04/2022,0.00,0.0,0.0,14.0,940.0,0.0,14.0,...,0.0,0.0,0.0,0.0,336.0,0.0,494.0,0.0,0.0,0.0


# Planned

In [20]:
Planned_excel =pd.read_excel("C:\\Users\\nadiajebin\\Desktop\\Hourly Dashboard\\Final\\planned_Data_10th_1_9.xlsx")
Planned_excel
plan=Planned_excel[['BusinessUnit','LineNumber','plandate','Sum of planqty','Average of SMV']].round(2)
plan['LineNumber'] = plan['LineNumber'].str.replace('Line-','').astype(int)
plan['Date'] = plan['plandate'].str.replace('-Apr','/04/2022')
plan['Date'] = plan['Date'].str.replace('1/04/2022','01/04/2022')
plan['Date'] = plan['Date'].str.replace('2/04/2022','02/04/2022')
plan['Date'] = plan['Date'].str.replace('3/04/2022','03/04/2022')
plan['Date'] = plan['Date'].str.replace('4/04/2022','04/04/2022')
plan['Date'] = plan['Date'].str.replace('5/04/2022','05/04/2022')
plan['Date'] = plan['Date'].str.replace('6/04/2022','06/04/2022')
plan['Date'] = plan['Date'].str.replace('7/04/2022','07/04/2022')
plan['Date'] = plan['Date'].str.replace('8/04/2022','08/04/2022')
plan['Date'] = plan['Date'].str.replace('9/04/2022','09/04/2022')

del plan['plandate']
plan=plan[['Date','BusinessUnit','LineNumber', 'Sum of planqty', 'Average of SMV']]

plan['Planned SAH']=(plan['Sum of planqty']*plan['Average of SMV'])/60
plan

,Date,BusinessUnit,LineNumber,Sum of planqty,Average of SMV,Planned SAH
0,01/04/2022,Unit-1,1,4305,2.14,153.545000
1,02/04/2022,Unit-1,1,5383,2.14,191.993667
2,03/04/2022,Unit-1,1,5383,2.14,191.993667
3,04/04/2022,Unit-1,1,5383,2.14,191.993667
4,05/04/2022,Unit-1,1,5383,2.14,191.993667
...,...,...,...,...,...,...
850,04/04/2022,Unit-4,9,809,12.54,169.081000
851,05/04/2022,Unit-4,9,809,12.54,169.081000
852,06/04/2022,Unit-4,9,809,12.54,169.081000
853,07/04/2022,Unit-4,9,809,12.54,169.081000


# Merge Plan and Forecast-Actual

Line Wise 

In [21]:
Prod_Fore_plan_line=pd.merge(plan, Prod_Fore2_line,on=['BusinessUnit','LineNumber','Date'],how='outer')
Prod_Fore_plan_line['Planned EFF']=(Prod_Fore_plan_line['Planned SAH']*100)/(10*24)
Prod_Fore_plan_line=Prod_Fore_plan_line[['Date', 'BusinessUnit', 'LineNumber','Total Defect', 'Total Garments', 'Reject', 'Rework',
                    'SMV', 'Working_Hour', 'PresentCadre','Sum of planqty', 'Planned SAH','Planned EFF',
                    'ForecastPcs','ForecastSAH', 'ForecastEff', 'Ok', 'Actual SAH', 'Actual EFF',
                    '6am-7am', '7am-8am', '8am-9am', '9am-10am', '10am-11am', '11am-12am',
                    '12pm-1pm', '1pm-2pm', '2pm-3pm', '3pm-4pm', '4pm-5pm', '5pm-6pm','8pm-9pm', '9pm-10pm']]
# '6pm-7pm', '11pm-12pm'
Prod_Fore_plan_line=Prod_Fore_plan_line.round(2)
Prod_Fore_plan_line=Prod_Fore_plan_line.fillna(0)
Prod_Fore_plan_line
excel1 = Prod_Fore_plan_line.to_excel('C:\\Users\\nadiajebin\\Desktop\\send\\Line Wise Report.xlsx')

Unit WIse

In [22]:
plan_unit=plan.copy()

plan_unit=plan_unit.groupby(['Date','BusinessUnit']).agg({'Sum of planqty':'sum', 'Planned SAH':'sum'}).reset_index()
plan_unit['Planned EFF']=(plan_unit['Planned SAH']*100)/(24*10*26)

Prod_Fore_plan_Unit=pd.merge(plan_unit, Prod_Fore3_sum,on=['BusinessUnit','Date'],how='outer')
Prod_Fore_plan_Unit=Prod_Fore_plan_Unit.round(2)
Prod_Fore_plan_Unit=Prod_Fore_plan_Unit[['Date', 'BusinessUnit','Sum of planqty','Working_Hour','Planned SAH', 'Planned EFF',
       'ForecastPcs', 'ForecastSAH', 'Forecast EFF', 'Ok','Actual SAH','Actual EFF','6am-7am', '7am-8am', '8am-9am',
       '9am-10am', '10am-11am','11am-12am', '12pm-1pm', '1pm-2pm', '2pm-3pm', '3pm-4pm', '4pm-5pm',
       '5pm-6pm', '8pm-9pm']]
Prod_Fore_plan_Unit=Prod_Fore_plan_Unit.fillna(0)

excel2 = Prod_Fore_plan_Unit.to_excel('C:\\Users\\nadiajebin\\Desktop\\send\\Unit Wise Plan Report.xlsx')

# Report With STYLE, PO,COLOR

Access Production API and Convert Data

In [24]:
production_url='http://10.12.13.164:8110/api/BasicQMSData/BirichinaDataForReport?StartDate=2022-04-02&EndDate=2022-04-12'
response = urlopen(production_url)
production_data_json = json.loads(response.read())
productionapi=pd.DataFrame(production_data_json)


In [25]:
raw_api=productionapi.copy()
########### Convert Time ###############
raw_api['Time']= pd.to_datetime(raw_api['Time'])
raw_api['Time']=raw_api['Time'].apply(lambda t: t.strftime('%H'))
raw_api['Time']=raw_api['Time'].astype(str)
########## Convert BatchQTY ############
raw_api['BatchQty']=raw_api['BatchQty'].astype(int)
########## Convert DefectCount ############
raw_api['DefectCount']=raw_api['DefectCount'].astype(int)
########## Convert SMV ############
raw_api['SMV']=raw_api['SMV'].astype(float)
########## Convert LineNumber Into Integer #########
raw_api['LineNumber'] = raw_api['LineNumber'].str.replace('Line-','').astype(int)

raw_api

,QmsDataID,BusinessUnit,LineNumber,Time,Date,BatchQty,BuyerName,ProductType,StyleSubCat,PoNumber,...,StyleCat,DefectID,OperatorId,MachineId,UserID,ModuleName,Shift,ShiftDate,OrderQty,Destination
0,0,Unit-1,1,07,06/04/2022,50,MS,None,KC.T61-MS-KPK-4970F-5PP PRINTED CL FB-GREEN MI...,2012024404,...,None,,None,None,80004,Sewing,Night Shift,4/5/2022 12:00:00 AM,None,None
1,0,Unit-1,1,07,06/04/2022,50,MS,None,KC.T61-MS-KPK-4970F-5PP PRINTED CL FB-GREEN MI...,2012024404,...,None,,None,None,80004,Sewing,Night Shift,4/5/2022 12:00:00 AM,None,None
2,0,Unit-1,1,07,06/04/2022,50,MS,None,KC.T61-MS-KPK-4970F-5PP PRINTED CL FB-GREEN MI...,2012024404,...,None,,None,None,80004,Sewing,Night Shift,4/5/2022 12:00:00 AM,None,None
3,0,Unit-1,1,07,06/04/2022,50,MS,None,KC.T61-MS-KPK-4970F-5PP PRINTED CL FB-GREEN MI...,2012024404,...,None,,None,None,80569,Sewing,Night Shift,4/5/2022 12:00:00 AM,None,None
4,0,Unit-1,1,07,06/04/2022,50,MS,None,KC.T61-MS-KPK-4970F-5PP PRINTED CL FB-GREEN MI...,2012024404,...,None,,None,None,80569,Sewing,Night Shift,4/5/2022 12:00:00 AM,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1167517,0,Unit-4,9,11,04/04/2022,30,HM,None,HM-JUNI NORA-S6 WITH LINING,304282-1339,...,None,,None,None,7477,Sewing,Day Shift,4/4/2022 12:00:00 AM,None,None
1167518,0,Unit-4,9,13,03/04/2022,12,HM,None,HM-JUNI NORA-S6 WITH LINING,304282-1339,...,None,,None,None,7477,Sewing,Day Shift,4/3/2022 12:00:00 AM,None,None
1167519,0,Unit-4,9,13,03/04/2022,12,HM,None,HM-JUNI NORA-S6 WITH LINING,304282-1339,...,None,,None,None,7477,Sewing,Day Shift,4/3/2022 12:00:00 AM,None,None
1167520,0,Unit-4,9,14,04/04/2022,30,HM,None,HM-JUNI NORA-S6 WITH LINING,304282-1339,...,None,,None,None,7477,Sewing,Day Shift,4/4/2022 12:00:00 AM,None,None


In [26]:
raw_api=raw_api.fillna(0)
raw_api2=raw_api[['BusinessUnit', 'LineNumber','Time', 'Date','StyleSubCat', 'PoNumber','Color','GarmentsNumber',
       'Size','DefectName', 'DefectType', 'DefectCount']]
raw_api2

raw_api_calc=raw_api2.copy()

######################################## Total ############################################
mac=raw_api_calc.copy()
mac=mac.groupby(['Date','BusinessUnit','LineNumber','StyleSubCat', 'PoNumber','Color','Size']).agg({'GarmentsNumber':'nunique','DefectCount' :'sum'}).reset_index()
mac['Total Defect']=mac['DefectCount']
mac['Total Garments']=mac['GarmentsNumber']
del mac['DefectCount']
del mac['GarmentsNumber']
mac

############################################## Counting Ok Defect type ############################
mav=raw_api_calc.copy()
mav=mav[mav['DefectType']=='Ok']
mav=mav.groupby(['Date','BusinessUnit','LineNumber','StyleSubCat', 'PoNumber','Color','Size']).agg({'DefectCount' :'sum','DefectType':'count'}).reset_index()
mav['Ok']=mav['DefectType']
del mav['DefectType']
del mav['DefectCount']
mav
############################################## Counting Reject Defect type ############################
mav1=raw_api_calc.copy()
mav1=mav1[mav1['DefectType']=='Reject']
mav1=mav1.groupby(['Date','BusinessUnit','LineNumber','StyleSubCat', 'PoNumber','Color','Size']).agg({'DefectCount' :'sum','DefectType':'count'}).reset_index()
mav1['Reject']=mav1['DefectType']
del mav1['DefectType']
del mav1['DefectCount']
mav1
############################################## Counting Rework Defect type ############################
mav2=raw_api_calc.copy()
mav2=mav2[mav2['DefectType']=='Rework']
mav2=mav2.groupby(['Date','BusinessUnit','LineNumber','StyleSubCat', 'PoNumber','Color','Size']).agg({'DefectCount' :'sum','DefectType':'count'}).reset_index()
mav2['Rework']=mav2['DefectType']
del mav2['DefectType']
del mav2['DefectCount']
mav2

############################################## Merging All Three Tables ############################
mao=pd.merge(mac,mav,on=['Date','BusinessUnit','LineNumber','StyleSubCat', 'PoNumber','Color','Size'],how='outer')
mao
mao1=pd.merge(mao,mav1,on=['Date','BusinessUnit','LineNumber','StyleSubCat', 'PoNumber','Color','Size'],how='outer')
mao1
mao2=pd.merge(mao1,mav2,on=['Date','BusinessUnit','LineNumber','StyleSubCat', 'PoNumber','Color','Size'],how='outer')

mao2=mao2.fillna(0)


############################################## Calculation and Data Type Conversion ############################
mao2['NewTotalGarments']=mao2['Total Garments']-(mao2['Reject']+mao2['Rework'])
mao2['Reject']=mao2['Reject'].astype(int)
mao2['Rework']=mao2['Rework'].astype(int)
mao2['Ok']=mao2['Ok'].astype(int)
mao2['NewTotalGarments']=mao2['NewTotalGarments'].astype(int)
del mao2['NewTotalGarments']
mao2=mao2[['Date','BusinessUnit','LineNumber','StyleSubCat', 'PoNumber','Color','Size','Total Defect','Total Garments','Ok', 'Reject', 'Rework']]
mao2

############################################## Pivoting Time  #####################################################
prod_time=raw_api.copy()
prod_time=prod_time.groupby(['Date','BusinessUnit', 'LineNumber','StyleSubCat', 'PoNumber',
                             'Color','Size','Time']).agg({'GarmentsNumber':'nunique'}).reset_index()
prod_time= prod_time.pivot_table('GarmentsNumber', ['Date','BusinessUnit', 'LineNumber','StyleSubCat',
                                                    'PoNumber','Color','Size'], 'Time').reset_index()
prod_time=prod_time.fillna(0)
prod_time

############################################## Rename Hour Column  #####################################################
prod_time['6am-7am']  =prod_time['06'].astype(int)
prod_time['7am-8am']  =prod_time['07'].astype(int)
prod_time['8am-9am']  =prod_time['08'].astype(int)
prod_time['9am-10am'] =prod_time['09'].astype(int)
prod_time['10am-11am']=prod_time['10'].astype(int)
prod_time['11am-12am']=prod_time['11'].astype(int)
prod_time['12pm-1pm'] =prod_time['12'].astype(int)
prod_time['1pm-2pm']  =prod_time['13'].astype(int)
prod_time['2pm-3pm']  =prod_time['14'].astype(int)
prod_time['3pm-4pm']  =prod_time['15'].astype(int)
prod_time['4pm-5pm']  =prod_time['16'].astype(int)
prod_time['5pm-6pm']  =prod_time['17'].astype(int)
# prod_time['6pm-7pm']  =prod_time['18'].astype(int)
prod_time['8pm-9pm']  =prod_time['19'].astype(int)
prod_time['9pm-10pm'] =prod_time['20'].astype(int)
# prod_time['11pm-12pm']=prod_time['21'].astype(int)

del prod_time['06']
del prod_time['07']
del prod_time['08']
del prod_time['09']
del prod_time['10']
del prod_time['11']
del prod_time['12']
del prod_time['13']
del prod_time['14']
del prod_time['15']
del prod_time['16']
del prod_time['17']
# del prod_time['18']
del prod_time['19']
del prod_time['20']
# del prod_time['21']

prod_merge=pd.merge(mao2,prod_time,on=['Date','BusinessUnit', 'LineNumber','StyleSubCat',
                                                    'PoNumber','Color','Size'], how='outer')
prod_merge

,Date,BusinessUnit,LineNumber,StyleSubCat,PoNumber,Color,Size,Total Defect,Total Garments,Ok,...,10am-11am,11am-12am,12pm-1pm,1pm-2pm,2pm-3pm,3pm-4pm,4pm-5pm,5pm-6pm,8pm-9pm,9pm-10pm
0,02/04/2022,Unit-1,1,KC.T61-MS-KPK-4970F-5PP PRINTED CL FB-GREEN MI...,2012024404,BLACK 10005,16,2,33,31,...,0,33,0,0,0,0,0,0,0,0
1,02/04/2022,Unit-1,1,KC.T61-MS-KPK-4970F-5PP PRINTED CL FB-GREEN MI...,2012024404,KPS MICRO STAR WHITE,16,0,1183,1183,...,51,102,0,0,294,736,0,0,0,0
2,02/04/2022,Unit-1,1,KC.T61-MS-KPK-4970F-5PP PRINTED CL FB-GREEN MI...,2012024404,KPS POLKA SPOT LIGHT ROSE,16,1,84,81,...,0,0,0,0,0,0,0,0,0,0
3,02/04/2022,Unit-1,1,KC.T61-MS-KPK-4970F-5PP PRINTED CL FB-GREEN MI...,2012024404,KPS SPOT GREEN,16,3,4466,4462,...,728,33,0,0,401,2800,56,448,0,0
4,02/04/2022,Unit-1,2,KC.T61-MS-KPK-2112X-IBO 5PK CL MIDI-ORANGE MIX...,2030909470,Bright Orange 22855,12,0,484,484,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1141,12/04/2022,Unit-3,30,KC.T61-MS-KPK-5024L-5PK LACE WAIST HL-FRESH BL...,2012027002,FRESH BLUE WHITE HEART,10,0,1008,1008,...,0,0,96,0,0,816,0,0,0,0
1142,12/04/2022,Unit-4,12,HM-JUNI NORA-S6 WITH LINING,307865-1339,35-109,75D,0,37,37,...,0,0,0,0,37,0,0,0,0,0
1143,12/04/2022,Unit-4,16,KC.T33 3006P- 3 PK NW Ctn FC DARK GREY MIX,TBA 2 2012037352,ANTIQUE ROSE - 21564,38B,2,132,130,...,0,0,0,0,22,0,0,0,0,0
1144,12/04/2022,Unit-4,16,KC.T33 3006P- 3 PK NW Ctn FC DARK GREY MIX,TBA 2 2012037352,CREAM 81001,36DD,12,808,788,...,0,0,0,0,314,0,494,0,0,0


Forecast Data

In [27]:
Forecast_excel =pd.read_excel("C:\\Users\\nadiajebin\\Desktop\\Hourly Dashboard\\Final\\Forecast_10th_April.xlsx")
forecast=Forecast_excel[['ProductionUnit','LineNumber', 'Date','PresentCadre','ForecastPcs',
                   'ForecastSAH','ForecastEff','Working_Hour','SMV']]
forecast['LineNumber'] = forecast['LineNumber'].str.replace('Line-','').astype(int)
forecast['Date']=forecast['Date'].dt.strftime('%d/%m/%Y')
forecast4apr=forecast.copy()
forecast4apr['BusinessUnit']=forecast4apr['ProductionUnit']
del forecast4apr['ProductionUnit']
forecast4apr=forecast4apr[['BusinessUnit','LineNumber','Date','PresentCadre','ForecastPcs',
                           'ForecastSAH','ForecastEff','Working_Hour','SMV']]
forecast4apr=forecast4apr.fillna(0)
forecast4apr=forecast4apr.round(2)
forecast4apr

C:\Users\NADIAJ~1\AppData\Local\Temp/ipykernel_9836/1415248643.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast['LineNumber'] = forecast['LineNumber'].str.replace('Line-','').astype(int)
C:\Users\NADIAJ~1\AppData\Local\Temp/ipykernel_9836/1415248643.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast['Date']=forecast['Date'].dt.strftime('%d/%m/%Y')


,BusinessUnit,LineNumber,Date,PresentCadre,ForecastPcs,ForecastSAH,ForecastEff,Working_Hour,SMV
0,Unit-2,1,04/04/2022,24,2497.0,207.25,86,10,4.98
1,Unit-2,2,04/04/2022,25,4044.0,192.09,76,10,2.85
2,Unit-2,3,04/04/2022,24,3324.0,214.95,89,10,3.88
3,Unit-2,4,04/04/2022,25,4349.0,216.00,86,10,2.98
4,Unit-2,5,04/04/2022,25,3137.0,149.01,59,10,2.85
...,...,...,...,...,...,...,...,...,...
765,Unit-4,23,10/04/2022,29,700.0,176.40,55,10,15.12
766,Unit-4,24,10/04/2022,26,1000.0,209.00,73,10,12.54
767,Unit-4,25,10/04/2022,31,880.0,208.56,61,10,14.22
768,Unit-4,26,10/04/2022,26,1100.0,211.02,73,10,11.51


Merge Forecast and Production

In [28]:
Prod_Fore2=pd.merge(forecast4apr, prod_merge,on=['BusinessUnit','LineNumber','Date'],how='outer')
Prod_Fore2=Prod_Fore2.fillna(0)
Prod_Fore2

Prod_Fore2
Prod_Fore3=Prod_Fore2.copy()
Prod_Fore2['Actual SAH']=(Prod_Fore2['SMV']*Prod_Fore2['Ok'])/60
Prod_Fore2['Actual EFF']=(Prod_Fore2['Actual SAH']*100)/(Prod_Fore2['Working_Hour']*Prod_Fore2['PresentCadre'])
Prod_Fore2=Prod_Fore2.round(2)
Prod_Fore2=Prod_Fore2.fillna(0)
Prod_Fore2.columns

Prod_Fore3_sum=Prod_Fore2.groupby(['BusinessUnit','Date','StyleSubCat',
       'PoNumber', 'Color', 'Size']).agg({'PresentCadre':'sum', 'ForecastPcs':'sum',
       'ForecastSAH':'sum','Working_Hour':'sum','Actual SAH':'sum', 'Total Defect':'sum',
       'Total Garments':'sum', 'Ok':'sum', 'Reject':'sum', 'Rework':'sum','6am-7am':'sum',
       '7am-8am':'sum', '8am-9am':'sum', '9am-10am':'sum', '10am-11am':'sum', '11am-12am':'sum', '12pm-1pm':'sum',
       '1pm-2pm':'sum', '2pm-3pm':'sum', '3pm-4pm':'sum', '4pm-5pm':'sum', '5pm-6pm':'sum', '8pm-9pm':'sum'}).reset_index()


Prod_Fore3_sum['Actual EFF']=(Prod_Fore3_sum['Actual SAH']*100)/(Prod_Fore3_sum['Working_Hour']*Prod_Fore3_sum['PresentCadre'])
Prod_Fore3_sum['Forecast EFF']=(Prod_Fore3_sum['ForecastSAH']*100)/(Prod_Fore3_sum['Working_Hour']*Prod_Fore3_sum['PresentCadre'])

Prod_Fore3_sum=Prod_Fore3_sum.round(2)
Prod_Fore3_sum=Prod_Fore3_sum.fillna(0)
Prod_Fore4_sum=Prod_Fore3_sum.copy()
Prod_Fore4_sum=Prod_Fore4_sum[['BusinessUnit', 'Date', 'StyleSubCat', 'PoNumber', 'Color', 'Size','Total Defect','Reject',
       'Rework', 'Working_Hour', 'PresentCadre', 'ForecastPcs', 'ForecastSAH','Forecast EFF','Ok','Actual SAH','Actual EFF', 
       '6am-7am', '7am-8am', '8am-9am', '9am-10am', '10am-11am','11am-12am', '12pm-1pm', '1pm-2pm', '2pm-3pm', '3pm-4pm', 
       '4pm-5pm','5pm-6pm', '8pm-9pm']]
Prod_Fore4_sum=Prod_Fore4_sum.fillna(0)
Prod_Fore4_sum

,BusinessUnit,Date,StyleSubCat,PoNumber,Color,Size,Total Defect,Reject,Rework,Working_Hour,...,9am-10am,10am-11am,11am-12am,12pm-1pm,1pm-2pm,2pm-3pm,3pm-4pm,4pm-5pm,5pm-6pm,8pm-9pm
0,Unit-1,02/04/2022,HM-FIONA 4PK MICRO HIPSTER -S6 SOLID,303058-1336,12-324.,M,54.0,4.0,13.0,0.0,...,0.0,764.0,905.0,0.0,136.0,800.0,240.0,48.0,40.0,0.0
1,Unit-1,02/04/2022,HM-FIONA 4PK MICRO HIPSTER -S6 SOLID,303058-1336,12-324.,S,5.0,1.0,1.0,0.0,...,0.0,48.0,48.0,97.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Unit-1,02/04/2022,HM-FIONA 4PK MICRO HIPSTER -S6 SOLID,303058-1336,72-107,M,16.0,5.0,61.0,0.0,...,0.0,862.0,1253.0,0.0,0.0,750.0,1255.0,1500.0,291.0,0.0
3,Unit-1,02/04/2022,HM-FIONA 4PK MICRO HIPSTER -S6 SOLID,303058-1336,72-107,S,6.0,2.0,5.0,0.0,...,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,302.0,0.0
4,Unit-1,02/04/2022,HM-FIONA 4PK MICRO HIPSTER -S6 SOLID,303058-1336,74-108,L,148.0,2.0,17.0,0.0,...,877.0,88.0,685.0,314.0,600.0,160.0,681.0,40.0,440.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1056,Unit-4,11/04/2022,KC.T33 3006P- 3 PK NW Ctn FC DARK GREY MIX,TBA 2 2012037352,ANTIQUE ROSE - 21564,38B,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,388.0,2.0,0.0
1057,Unit-4,12/04/2022,HM-JUNI NORA-S6 WITH LINING,307865-1339,35-109,75D,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,37.0,0.0,0.0,0.0,0.0
1058,Unit-4,12/04/2022,HM-JUNI NORA-S6 WITH LINING,307865-1339,35-109,80B,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0
1059,Unit-4,12/04/2022,KC.T33 3006P- 3 PK NW Ctn FC DARK GREY MIX,TBA 2 2012037352,ANTIQUE ROSE - 21564,38B,2.0,0.0,2.0,0.0,...,110.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0


Plan DATA

In [29]:
Planned_excel =pd.read_excel("C:\\Users\\nadiajebin\\Desktop\\Hourly Dashboard\\Final\\planned_Data_10th_1_9.xlsx")
Planned_excel
plan=Planned_excel[['BusinessUnit','LineNumber','plandate','Sum of planqty','Average of SMV']].round(2)
########## Convert LineNumber Into Integer #########
plan['LineNumber'] = plan['LineNumber'].str.replace('Line-','').astype(int)

plan['Date'] = plan['plandate'].str.replace('-Apr','/04/2022')
plan['Date'] = plan['Date'].str.replace('1/04/2022','01/04/2022')
plan['Date'] = plan['Date'].str.replace('2/04/2022','02/04/2022')
plan['Date'] = plan['Date'].str.replace('3/04/2022','03/04/2022')
plan['Date'] = plan['Date'].str.replace('4/04/2022','04/04/2022')
plan['Date'] = plan['Date'].str.replace('5/04/2022','05/04/2022')
plan['Date'] = plan['Date'].str.replace('6/04/2022','06/04/2022')
plan['Date'] = plan['Date'].str.replace('7/04/2022','07/04/2022')
plan['Date'] = plan['Date'].str.replace('8/04/2022','08/04/2022')
plan['Date'] = plan['Date'].str.replace('9/04/2022','09/04/2022')

del plan['plandate']
plan=plan[['Date','BusinessUnit','LineNumber', 'Sum of planqty', 'Average of SMV']]

plan['Planned SAH']=(plan['Sum of planqty']*plan['Average of SMV'])/60
plan

,Date,BusinessUnit,LineNumber,Sum of planqty,Average of SMV,Planned SAH
0,01/04/2022,Unit-1,1,4305,2.14,153.545000
1,02/04/2022,Unit-1,1,5383,2.14,191.993667
2,03/04/2022,Unit-1,1,5383,2.14,191.993667
3,04/04/2022,Unit-1,1,5383,2.14,191.993667
4,05/04/2022,Unit-1,1,5383,2.14,191.993667
...,...,...,...,...,...,...
850,04/04/2022,Unit-4,9,809,12.54,169.081000
851,05/04/2022,Unit-4,9,809,12.54,169.081000
852,06/04/2022,Unit-4,9,809,12.54,169.081000
853,07/04/2022,Unit-4,9,809,12.54,169.081000


Unit Wise Report

In [33]:
plan_unit=plan.copy()

plan_unit=plan_unit.groupby(['Date','BusinessUnit']).agg({'Sum of planqty':'sum', 'Planned SAH':'sum'}).reset_index()
plan_unit['Planned EFF']=(plan_unit['Planned SAH']*100)/(24*10*26)

Prod_Fore_plan_Unit=pd.merge(plan_unit, Prod_Fore3_sum,on=['BusinessUnit','Date'],how='outer')
Prod_Fore_plan_Unit=Prod_Fore_plan_Unit.round(2)
Prod_Fore_plan_Unit=Prod_Fore_plan_Unit.fillna(0)

Prod_Fore_plan_Unit['Planned Pcs']=Prod_Fore_plan_Unit['Sum of planqty'].astype(int)
del Prod_Fore_plan_Unit['Sum of planqty']
Prod_Fore_plan_Unit['ForecastPcs']=Prod_Fore_plan_Unit['ForecastPcs'].astype(int)
Prod_Fore_plan_Unit['Ok']=Prod_Fore_plan_Unit['Ok'].astype(int)
Prod_Fore_plan_Unit['Working_Hour']  =Prod_Fore_plan_Unit['Working_Hour'].astype(int)

Prod_Fore_plan_Unit=Prod_Fore_plan_Unit[['Date', 'BusinessUnit','StyleSubCat', 'PoNumber', 'Color', 'Size','Planned Pcs','Working_Hour','Planned SAH', 'Planned EFF',
       'ForecastPcs', 'ForecastSAH', 'Forecast EFF', 'Ok','Actual SAH','Actual EFF','6am-7am', '7am-8am', '8am-9am',
       '9am-10am', '10am-11am','11am-12am', '12pm-1pm', '1pm-2pm', '2pm-3pm', '3pm-4pm', '4pm-5pm',
       '5pm-6pm', '8pm-9pm']]
Prod_Fore_plan_Unit.iloc[:,16:]  =Prod_Fore_plan_Unit.iloc[:,16:].astype(int)

Prod_Fore_plan_Unit.iloc[:,12:]



excel3 = Prod_Fore_plan_Unit.to_excel('C:\\Users\\nadiajebin\\Desktop\\send\\Unit Wise Plan Report with Style.xlsx')

Line Wise Report

In [35]:
Prod_Fore2_line=Prod_Fore2[['BusinessUnit', 'LineNumber', 'Date','StyleSubCat','PoNumber', 'Color','Size', 'SMV','Working_Hour',
                'PresentCadre','Total Defect','Reject', 'Rework','ForecastPcs','ForecastSAH', 'ForecastEff',
                'Ok','Actual SAH', 'Actual EFF','6am-7am', '7am-8am','8am-9am', '9am-10am', '10am-11am',
                '11am-12am', '12pm-1pm', '1pm-2pm','2pm-3pm', '3pm-4pm', '4pm-5pm', '5pm-6pm', '8pm-9pm','9pm-10pm']]
# '6pm-7pm','11pm-12pm'
Prod_Fore2_line
Prod_Fore_plan_line=pd.merge(plan, Prod_Fore2_line,on=['BusinessUnit','LineNumber','Date'],how='outer')
Prod_Fore_plan_line
Prod_Fore_plan_line['Planned EFF']=(Prod_Fore_plan_line['Planned SAH']*100)/(10*24)
Prod_Fore_plan_line=Prod_Fore_plan_line.round(2)
Prod_Fore_plan_line=Prod_Fore_plan_line.fillna(0)
Prod_Fore_plan_line['Planned Pcs']=Prod_Fore_plan_line['Sum of planqty'].astype(int)
del Prod_Fore_plan_line['Sum of planqty']
Prod_Fore_plan_line['Total Defect']=Prod_Fore_plan_line['Total Defect'].astype(int)
Prod_Fore_plan_line['Reject']=Prod_Fore_plan_line['Reject'].astype(int)
Prod_Fore_plan_line['Rework']=Prod_Fore_plan_line['Rework'].astype(int)
Prod_Fore_plan_line['Working_Hour']=Prod_Fore_plan_line['Working_Hour'].astype(int)
Prod_Fore_plan_line['ForecastPcs']=Prod_Fore_plan_line['ForecastPcs'].astype(int)
Prod_Fore_plan_line['Ok']=Prod_Fore_plan_line['Ok'].astype(int)

Prod_Fore_plan_line=Prod_Fore_plan_line[['Date', 'BusinessUnit', 'LineNumber','StyleSubCat','PoNumber', 'Color','Size',
                    'Total Defect','Reject', 'Rework','SMV', 'Working_Hour', 'PresentCadre',
                    'Planned Pcs', 'Planned SAH','Planned EFF',
                    'ForecastPcs','ForecastSAH', 'ForecastEff', 
                    'Ok', 'Actual SAH', 'Actual EFF',
                    '6am-7am', '7am-8am', '8am-9am', '9am-10am', '10am-11am', '11am-12am','12pm-1pm', '1pm-2pm', '2pm-3pm',
                    '3pm-4pm', '4pm-5pm', '5pm-6pm','8pm-9pm', '9pm-10pm']]
# '6pm-7pm', '11pm-12pm'
Prod_Fore_plan_line.iloc[:,22:]=Prod_Fore_plan_line.iloc[:,22:].astype(int)

Prod_Fore_plan_line.iloc[:,17:]

excel4 = Prod_Fore_plan_line.to_excel('C:\\Users\\nadiajebin\\Desktop\\send\\Line Wise Report with Style.xlsx')

In [36]:
Prod_Fore2_line=Prod_Fore2[['BusinessUnit', 'LineNumber', 'Date','StyleSubCat','PoNumber', 'Color','Size', 'SMV','Working_Hour',
                'PresentCadre','Total Defect','Reject', 'Rework','ForecastPcs','ForecastSAH', 'ForecastEff',
                'Ok','Actual SAH', 'Actual EFF','6am-7am', '7am-8am','8am-9am', '9am-10am', '10am-11am',
                '11am-12am', '12pm-1pm', '1pm-2pm','2pm-3pm', '3pm-4pm', '4pm-5pm', '5pm-6pm', '8pm-9pm','9pm-10pm']]
# '6pm-7pm','11pm-12pm'
Prod_Fore2_line
Prod_Fore_plan_line=pd.merge(plan, Prod_Fore2_line,on=['BusinessUnit','LineNumber','Date'],how='outer')
Prod_Fore_plan_line
Prod_Fore_plan_line['Planned EFF']=(Prod_Fore_plan_line['Planned SAH']*100)/(10*24)
Prod_Fore_plan_line=Prod_Fore_plan_line.round(2)
Prod_Fore_plan_line=Prod_Fore_plan_line.fillna(0)
Prod_Fore_plan_line['Planned Pcs']=Prod_Fore_plan_line['Sum of planqty'].astype(int)
del Prod_Fore_plan_line['Sum of planqty']
Prod_Fore_plan_line['Total Defect']=Prod_Fore_plan_line['Total Defect'].astype(int)
Prod_Fore_plan_line['Reject']=Prod_Fore_plan_line['Reject'].astype(int)
Prod_Fore_plan_line['Rework']=Prod_Fore_plan_line['Rework'].astype(int)
Prod_Fore_plan_line['Working_Hour']=Prod_Fore_plan_line['Working_Hour'].astype(int)
Prod_Fore_plan_line['ForecastPcs']=Prod_Fore_plan_line['ForecastPcs'].astype(int)
Prod_Fore_plan_line['Ok']=Prod_Fore_plan_line['Ok'].astype(int)

Prod_Fore_plan_line=Prod_Fore_plan_line[['Date', 'BusinessUnit', 'LineNumber','StyleSubCat','PoNumber', 'Color','Size',
                    'Total Defect','Reject', 'Rework','SMV', 'Working_Hour', 'PresentCadre',
                    'Planned Pcs', 'Planned SAH','Planned EFF',
                    'ForecastPcs','ForecastSAH', 'ForecastEff', 
                    'Ok', 'Actual SAH', 'Actual EFF',
                    '6am-7am', '7am-8am', '8am-9am', '9am-10am', '10am-11am', '11am-12am','12pm-1pm', '1pm-2pm', '2pm-3pm',
                    '3pm-4pm', '4pm-5pm', '5pm-6pm','8pm-9pm', '9pm-10pm']]
# '6pm-7pm', '11pm-12pm'
Prod_Fore_plan_line.iloc[:,22:]=Prod_Fore_plan_line.iloc[:,22:].astype(int)

Prod_Fore_plan_line.iloc[:,17:]

excel4 = Prod_Fore_plan_line.to_excel('C:\\Users\\nadiajebin\\Desktop\\send\\Line Wise Report with Style.xlsx')

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Apr/2022 13:19:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2022 13:19:16] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2022 13:19:16] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2022 13:19:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2022 13:19:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2022 13:19:18] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Apr/2022 13:19:18] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Apr/2022 13:19:18] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2022 13:19:18] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2022 13:19:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2022 13:19:22]